## A. Import Intervals and Other Code

* The first step is to import all the code required for the Notebook
* **`arrow/run`** or **`Shift + Enter`** in the following cell:

In [381]:
import intervals
from intervals import * 
from intervals import main_objs
import intervals.visualizations as viz
import pandas as pd
import re
import altair as alt
import matplotlib.pyplot as plt
import seaborn as sns
from ipywidgets import interact
from pandas.io.json import json_normalize
from pyvis.network import Network
from IPython.display import display
import requests
import os

MYDIR = ("saved_csv")
CHECK_FOLDER = os.path.isdir(MYDIR)

# If folder doesn't exist, then create it.
if not CHECK_FOLDER:
    os.makedirs(MYDIR)
    print("created folder : ", MYDIR)
else:
    print(MYDIR, "folder already exists.")
    
MUSDIR = ("Music_Files")
CHECK_FOLDER = os.path.isdir(MUSDIR)

# If folder doesn't exist, then create it.
if not CHECK_FOLDER:
    os.makedirs(MUSDIR)
    print("created folder : ", MUSDIR)

else:
    print(MUSDIR, "folder already exists.")

saved_csv folder already exists.
Music_Files folder already exists.


## B. Importing Pieces

### B.1 Import a Single Piece and Check Metadata for Title and Composer

- Here you will want to select the appropriate 'prefix' that identifies the location of your file.
- `'Music_Files/'` is for files in the local notebook; `'https://crimproject.org/mei/'` is for the files on CRIM.
- Then provide the full name (and extension) of your music file, such as `'CRIM_Model_0038.mei'`

In [382]:
# Select a prefix:
# prefix = 'Music_Files/'
prefix = 'https://crimproject.org/mei/' 


# Add your filename here
mei_file = 'CRIM_Model_0008.mei'

# These join the strings and import the piece
url = prefix + mei_file
piece = importScore(url)

print(piece.metadata)

{'title': 'Ave Maria', 'composer': 'Josquin Des Prés', 'date': 1502}


In [389]:
# get the P-types, ngrams as series, and the notes with detail index
# must pass the same "n" to mel and ptypes
ngram_length = 5

p_types = piece.presentationTypes(melodic_ngram_length=ngram_length)
# get the ngrams as a series
notes = piece.notes()
notes_det = piece.detailIndex(notes, offset=True)
mel = piece.melodic(end=False)
ngrams = piece.ngrams(df=mel, n=ngram_length, offsets='both')
ngrams_series = ngrams.stack()

# notes
notes_det = notes_det.reset_index(level=['Measure', 'Beat'])
notes_det = notes_det[['Measure', 'Beat']]
notes_det.index.names = ['Offset']


In [390]:
# the function
def ngramfilter(row, ngrams_series, notes_det):
    offsets = row["Offsets"]
    voices = row["Voices"]
    pairs = list(zip(offsets, voices))

    ng_list = []
    for pair in pairs:
        offset = pair[0]
        voice = pair[1]
        ngram_coordinates = ngrams_series[(ngrams_series.index.get_level_values(0) == offset) & (ngrams_series.index.get_level_values(2) == voice)]
        ngram_coordinates = ngram_coordinates.reset_index()
        ngram_coordinates = ngram_coordinates.rename(columns={'level_2' : 'Voice'})
        ng_list.append(ngram_coordinates)
    ng_df = pd.concat(ng_list)
    ng_df_first = ng_df.set_index('First')
    ng_df_last = ng_df.set_index('Last')

    merged_df = ng_df_first.merge(notes_det, left_index=True, right_index=True, how='left')
    merged_df.rename(columns={"Measure" : "FirstMeasure", "Beat" : "FirstBeat"}, inplace=True)
    merged_df.reset_index(inplace=True)
    merged_df2 = ng_df_last.merge(notes_det, left_index=True, right_index=True, how='left')
    merged_df2.rename(columns={"Measure" : "LastMeasure", "Beat" : "LastBeat"}, inplace=True)
    merged_df2.reset_index(inplace=True)
    cols_to_use = merged_df2.columns.difference(merged_df.columns)
    final = pd.merge(merged_df, merged_df2[cols_to_use], left_index=True, right_index=True, how='outer')

    

    return final
    
             

In [394]:
p_types['result'] = p_types.apply(lambda row: ngramfilter(row, ngrams_series, notes_det), axis=1)

# sample result
for item in p_types['result']:
    print(item)
# p_types

   First  Last       Voice                      0  FirstMeasure  FirstBeat  \
0    0.0  24.0  [Superius]  (P4, P1, M2, M2, -M3)             1        1.0   
1   16.0  40.0       Altus  (P4, P1, M2, M2, -M3)             3        1.0   
2   32.0  56.0       Tenor  (P4, P1, M2, M2, -M3)             5        1.0   
3   48.0  72.0      Bassus  (P4, P1, M2, M2, -M3)             7        1.0   

   LastBeat  LastMeasure  
0       1.0            4  
1       1.0            6  
2       1.0            8  
3       1.0           10  
   First   Last       Voice                         0  FirstMeasure  \
0   56.0   74.0  [Superius]  (-m2, -M2, -M2, M2, -M2)             8   
1   72.0   91.0       Altus  (-m2, -M2, -M2, M2, -M2)            10   
2   88.0  106.0       Tenor  (-m2, -M2, -M2, M2, -M2)            12   

   FirstBeat  LastBeat  LastMeasure  
0        1.0       2.0           10  
1        1.0       2.5           12  
2        1.0       2.0           14  
   First   Last       Voice          